# Zhsegment

This homowork is on Chinese word segmentation. 
The F-score for bigram and smoothing solution is 0.92.

### 1. Implementation of Baseline System

We applied the Iterative Segmenter Algorithm to implement the baseline system by incorporating it into the Segment class and the Entry class of the file. Also, some classes and functions in homework 0 are reused in this task. Below are the Segment and Entry class:

In [67]:
class Segment:

    def __init__(self, Pw):
        self.Pw = Pw
    
    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        if len(text) == 1: return [text]
        ## Initialize the heap ##
        heap = []
        chart = [None] * len(text)
        for i in range(0,len(text)):
            word = text[:i+1]
            " TODO: set the maximum length of unknown word "
            if self.Pw.contains(word) or len(word) <= 1: 
                heappush(heap, Entry(word, 0, self.Pw(word)))

        ## Iteratively fill in chart[i] for all i ##
        while heap:
            entry = heappop(heap)
            #print(entry)
            #print(len(heap))
            endindex = len(entry.word) - 1 + entry.startpos
            if chart[endindex]:
                preventry = chart[endindex]
                if entry.logprob > preventry.logprob:
                    chart[endindex] = entry
                else:
                    continue
            else:
                chart[endindex] = entry
            for i in range(endindex+1,len(text)):
                word = text[endindex+1:i+1]
                "TODO: set the maximum length of unknown word "
                if self.Pw.contains(word) or len(word) <= 1:
                    newentry = Entry(word, endindex+1, entry.logprob + self.Pw(word), entry)
                    if not newentry in heap:
                        heappush(heap, newentry) 

        ## Get the best segmentation ##
        finalindex = len(text) - 1
        finalentry = chart[finalindex]
        segmentation = []
        while finalentry:
            segmentation.insert(0,finalentry.word)
            finalentry = finalentry.backptr
        return segmentation
    
class Entry:
    def __init__(self, word, startpos, logprob, backptr=None):
        self.word = word
        self.startpos = startpos
        self.logprob = logprob
        self.backptr = backptr
    
    def __lt__(self, other):
        return (self.startpos, self.logprob) < (other.startpos, other.logprob)

    def __eq__(self, other):
        return (self.word, self.startpos, self.logprob, self.backptr) == (other.word, other.startpos, other.logprob, other.backptr)

    def __str__(self):
        return 'Entry(word='+self.word+', logprob='+str(self.logprob)+ ')'

Two problems occur when we test and improve this implementation with dev.txt
1.Orignal alogrithm (improve score 0.89 to 0.92)
    
The orignal alogrithm in hw0 will produce missing lines dev.out which can only produce up to 0.89 in dev.out result. After we see this problem, we know that we need to add a initial probability to the heap when the entire line is unknown word in the initialization, so we added the probability to heap when the word length = 1, which is the beginning word of each sentence.  And we are able to improve the score from 0.89 to 0.92.
    
2.The running time is long.
   
So when pushing new entries, we set the limit of allowing the word with maximun length equal to 6 during the initializaion heap, since it is very unlikely to be a word for length more than 6 in known word after examing the "count_1w" and "count_2w"). So in later implementations (see below), we do not check a word with length larger than 6. 

3.Althought the score is 0.92 on unigram, but it may help on the improvement with jelinek mercer smoothing. We decided to combine the bigram model and smoothing technique to create another implementation as shown in Section 2.

we have tried the split out the some part of test.out (small test.out) for testing, and the original score (which has 0.92 score on dev.out) result a 0.76 on small test.txt. After we applied smoothing, tune the parameters lambda and set the word length to 2 which methoned is part 1 above, we are able get the results improved from 0.79 to 0.76, but dev.txt keeps at 0.92.
    
From result of 3, it makes sense that the improvement is significant on test.txt instead of dev.txt since the ideal of smoothing is to provide better overall improvements on unknown words. And, test.out characters should have more unknown than dev.txt

### 2. Improvement with Bigram and Smoothing


In [68]:
from zhsegment import * 

To use bigram, first we create a instance of Pdist class to get a probability distribution estimated from counts in "count_2w.txt" and "count_1w.txt"

In [69]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    def __call__(self, key): 
        if key in self: 
            return self[key]/(self.N *len(key)/2)
        else: 
            return self.missingfn(key, self.N)
        
PBi = Pdist(data=datafile("../data/count_2w.txt"), missingfn=returnZero)
PUi = Pdist(data=datafile("../data/count_1w.txt"), missingfn=reduce_probability_for_long_word)

Now, we can use PBi to compute a probability estimated from the dataset, below is an example:

In [70]:
PBi('发达 国家')

4.230068738617002e-05

In [71]:
PUi('发达')

0.00019975324599024735

Next, we create a function cPw(word, prev) to compute the conditional probability P(word | prev).

In [72]:
def cPw(word, prev):
    try:
       value = PBi(prev + ' ' + word)/float(PUi(prev))
       return value
    except KeyError:
       return PUi(word)

In [73]:
cPw('国家', '发达')

0.2117647058823529

Next we add the bigram to the Entry and Segment class:

In [74]:
class Entry:
# each entry in the chart has four components: Entry(word, start-position, log-probability, back-pointer)
# the back-pointer in each entry links it to a previous entry that it extends
    def __init__(self, word, start_position, log_probability, back_pointer=None):
        self.word = word
        self.start_position = start_position # start position
        self.log_probability = log_probability # log probability
        self.back_pointer = back_pointer # back_pointer
    def __lt__(self, other):
        return  self.log_probability < other.log_probability
    def __eq__(self, other):
        # return (self.word, self.start_position, self.log_probability, self.backptr) == (other.word, other.start_position, other.log_probability, other.back_pointer)

        return (self.word, self.start_position, self.log_probability, self.back_pointer) == (other.word, other.start_position, other.log_probability, other.back_pointer)
    def __str__(self):
        return 'Entry(word='+self.word+', logprob='+str(self.log_probability)+ ')'

class Segment:

    def __init__(self, PUi, PBi):
        self.PUi = PUi
        self.PBi = PBi
    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        # Initialize the heap 
        heap = [] # a list or priority queue containing the entries to be expanded, sorted on start-position or log-probability
        # for each word that matches input at position 0

        for i in range(min(len(text), maximum_word_length)):
            word = text[:i+1]
            if word in PUi or len(word) <= 2: # Check if word exists fron counts1w or Pw
                heappush(heap, Entry(word, 0, log10((1-lam)*PUi(word)))) #  insert Entry (word, 0, logPw(word), None) to heap

        chart = [None] * len(text)
        finalindex = 0
        while heap:
            entry = heappop(heap) # top entry in the heap
            endindex = len(entry.word) -1  + entry.start_position
           
            if finalindex < endindex :
                finalindex = endindex
            if chart[endindex]:  # if chart[endindex] exists entry
                preventry = chart[endindex]
                if entry.log_probability > preventry.log_probability: # then we check the current if has higher log probability
                    chart[endindex] = entry
                else:
                    continue
            else:  # if not previous entry for chart[endindex], chart[endindex] not exists
                chart[endindex] = entry # store current entry to chart[endindex]
                
            #for each newword that matches input starting at position endindex+1
            for i in range (endindex+1, min(len(text), maximum_word_length +endindex+1)):
                newword = text[endindex+1:i+1]
                try:
                    cPw = PBi[entry.word + ' ' + newword]/float(PUi[entry.word])
                except KeyError:
                    cPw = PUi(word)
                # print(cPw)
                newentry = Entry(newword, endindex+1, entry.log_probability + log10(lam*cPw+(1-lam)*PUi(newword)), entry)
                if not newentry in heap:
                    heappush(heap, newentry)

        #Get the best segmentation ##
        finalindex = len(text) - 1
        finalentry = chart[finalindex]
        segmentation = []

        while finalentry:
            segmentation.insert(0,finalentry.word)
            finalentry = finalentry.back_pointer
        return segmentation # return array of best segmentation, and " ".join used later to combine all array string items

Notice that we have introduced probability lam * cPw + (1-lam) * Pui(newword) as the probability as of a newentry, thus implementing the idea of smoothing, combining the probabilty of unigram and bigram of a word. The cPw is given as below: cPw = PBi[entry.word + ' ' + newword] / float(PUi[entry.word]). 

In order to find the lam in the smoothing equation, we tried diffrent values 

lam = 0.7125
dev.out score: 0.92

lam = 0.98
dev.out score: 0.90

lam = 0.9
dev.out score: 0.91

lam = 0.7
dev.out score: 0.92

......

We then set lam = 0.7125 and the score reachs 0.92 for dev.txt

Here is a look at the segmentation output:

In [ ]:
segmenter = Segment(PUi, PBi)
with open("../data/input/dev.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。
上午 在 这里 签字 的 是 知识 信息 网络 通讯 技术 和 脱氧 核 糖 核 酸 生物 技术 两 个 项目 ， 同时 还 签订 了 语言 教 学 交流 合作 协议 。
这 三 个 项目 是 分别 由 国务院 发展 研究 中心 国际 技术 经济 研究所 上海 分 所 和 上海市 浦东 继续 教育 中心 ， 与 美国 知识 信息 网络 公司 、 世界 学习 组织 、 海 赛克 公司 签订 的 。
知识 信息 网络 公司 主席 ＣＥ麦奈 特 在 签字 仪式 上 说 ， 中 美 合作 高 科技 项目 的 签订 ， 具有 重要 的 意义 。
他 还 期望 美 中 之间 建立 电子 高速 信息 公路 ， 使 纽约 股票 交易所 与 上海 证券 交易所 能 早日 实现 交易 及 信息 交流 。
世界 学习 组织 董事长 朱 迪·梅罗 希望 在 今年 底 与 上海 建立 一 个 合资 的 教育 机构 。
据 美方 人员 说 ， 今天 签订 的 三 个 项目 ， 得到 前来 上海 的 美国 商务 部长 布朗 的 关心 与 支持 。
（ 完 ）
中国 将 于 十二月 举办 “ 华 夏 艺术 节 ”
新华社 北京 九月 一日 电 （ 记者 高 建 新 ）
为 加强 海内外 炎黄子孙 的 文化 交流 和 联系 ， 中国 将 于 今年 十二月 八日 到 十八日 在 北京 和 深圳 两 地 同时 举办 “ 华 夏 艺术 节 ” 。
这 是 一 次 由 人民 团体 、 政府 文化 部门 和 企业 联手 举办 的 艺术 盛会 。
其 宗旨 是 ， 以 中华 民族 传统 文化 为 纽带 ， 联系 和 团结 海内外 同胞 ， 弘扬 民族 文化 ， 展示 祖国 优秀 艺术 的 魅力 ， 展示 华侨 、 华人 艺术 家 在 艺术 领域 和 世界 文化 交流 中 的 成就 和 风采 。
艺术 节 的 内容 主要 包括 隆重 的 开幕式 和 闭幕 式 ， 各 具 特色 的 大型 文艺 表 演 以及 美 术 、 摄 影 展览 。
据 介绍 ， 十二月 八日 在 深圳 体育馆 开幕 的 艺术 节 ， 将 集中 展示 二十

国务院 总理 李鹏 今天 在 中 南海 紫 光 阁 会见 中国 推行 公务 员 制度 经验 交流会 全体 代表 时 指出 ， 推行 公务 员 制度 是 中国 政治 体制 改革 的 一 项 重要 内容 ， 是 干部 人事 制度 的 重大 改革 ， 是 建立 社会主义 市场 经济 体制 的 客观 需要 ， 要 有 领导 、 有 步骤 地 加快 推行 步伐 。
这 次 推行 公务 员 制度 经验 交流会 是 昨天 开始 召开 的 ， 各 省 、 自治区 、 直辖市 人事 厅 局长 、 国务院 各 部 委 、 直属 机构 人事 部门 的 负责人 共 一百二十多 人 出席 了 会议 。
李鹏 肯定 了 一 年 来 国家 公务 员 制度 推行 工作 取得 的 成效 。
他 说 ， 我们 要 认真 总结 和 推广 这些 好 的 经验 ， 建立 起 激励 竞争 和 勤 政 廉政 机制 ， 建立 一 支 以 为 人民 服务 为 宗旨 、 密切 联系 群众 、 精干 高效 、 廉洁奉公 、 忠 于 职 守 的 国家 公务 员 队伍 ， 增强 政府 机关 的 生机 和 活力 。
李鹏 提出 ， 推行 公务 员 制度 ， 要 按照 《 国家 公务 员 暂行 条例 》 依法 办事 ， 不 能 有 随意性 。
要 把 这 项 工作 作为 政治 体制 改革 的 一 件 大 事 来 抓 ， 结合 改革 、 精 简 机构 来 推行 公务 员 制度 ； 要 形成 公务 员 的 新 陈 代 谢 机制 ， 使 青年 人才 不断 地 进入 到 公务 员 队伍 当 中 。
国务 委员 李 贵 鲜 、 罗干 参加 了 会见 。
（ 完 ）
马来西亚 副总理 结束 访 华
新华社 上海 八月 三十一日 电 （ 记者 周解蓉 ）
马来西亚 副总理 安 瓦 尔 今天 下午 结束 了 对 中国 的 访问 ， 离开 上海 前往 东京 。
安 瓦 尔 于 昨天 下午 自 南京 抵达 上海 。
昨晚 ， 上海市 副市长 赵启 正 代表 黄菊 市长 会见 并 宴 请 了 安 瓦 尔 一行 。
今天 上午 ， 安 瓦 尔 出席 了 闵行 中 马 合资 企业 奠基 仪式 ， 随后 参观 了 浦东 金 桥 出口 加工区 。
（ 完 ）
李鹏 会见 澳门 总 督韦奇立
新华社 北京 九月 一日 电
国务院 总理 李鹏 今天 在 这里 

### 3. Summary

In this homework, we have
 1. Implemented the baseline system
 2. Improve the baseline system with bigram and smoothing
 3. Reach a score of 0.92 for dev.txt and score 0.79 of small test.out